In [200]:
import numpy as np
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def test(runs = 1000):
    for i in range(runs):
        # generate random channel
        # generate train data
        # generate test data
        # gradient descent
        # test on output
        